In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import string
from nltk.corpus import stopwords
#open file
data = pd.read_csv('dresscode.csv',sep=';')
data['length'] = data['Product Name'].str.len()
#remove all puncs
def text_process(mess):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english') if word.lower() not in stopwords.words('dutch')]
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(data['Product Name'].values.astype('U'))
# Print total number of vocab words
messages_bow = bow_transformer.transform(data['Product Name'].values.astype('U'))
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(messages_tfidf, data['Category Path'].values.astype('U'))

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
msg_train, msg_test, label_train, label_test = \
train_test_split(data['Product Name'].values.astype('U'), data['Category Path'].values.astype('U'), test_size=0.1)
print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
#https://www.kdnuggets.com/2019/01/build-api-machine-learning-model-using-flask.html

485735 53971 539706


In [3]:
import pickle
pickle.dump(spam_detect_model, open('model.pickle', 'wb'))
pickle.dump(bow_transformer, open('bow_transformer.pickle', 'wb'))
pickle.dump(tfidf_transformer, open('tfidf_transformer.pickle', 'wb'))

In [4]:
from sklearn.metrics import classification_report
message4 = "Leren jack"
bow4 = bow_transformer.transform([message4])
tfidf4 = tfidf_transformer.transform(bow4)
counter = 0
predicted = spam_detect_model.predict_proba(tfidf4)
for x in spam_detect_model.classes_:
  proba  = round(predicted[0][counter],2)
  if proba > 0.01:
      print(x + ' probility '+ str(proba))
  counter +=1

#or show list
#list(zip(spam_detect_model.classes_, predicted[0]))
print(classification_report(predictions,label_test))

Dames%26gt%3BAccessoires%26gt%3BRiemen probility 0.02
Dames%26gt%3BAccessoires%26gt%3BTassen%26gt%3BHandtassen probility 0.02
Dames%26gt%3BAccessoires%26gt%3BTassen%26gt%3BSchoudertassen probility 0.02
Dames%26gt%3BKleding%26gt%3BBlazers probility 0.02
Dames%26gt%3BKleding%26gt%3BJassen%26gt%3BJacks probility 0.03
Dames%26gt%3BKleding%26gt%3BJassen%26gt%3BLeren+Jassen probility 0.16
Dames%26gt%3BKleding%26gt%3BJassen%26gt%3BMantels probility 0.04
Dames%26gt%3BKleding%26gt%3BJurken%26gt%3BCasual+Jurken probility 0.03
Dames%26gt%3BSchoenen%26gt%3BLaarzen%26gt%3BEnkellaarzen probility 0.03
Dames%26gt%3BSchoenen%26gt%3BLaarzen%26gt%3BKlassieke+Laarzen probility 0.02
Dames%26gt%3BSchoenen%26gt%3BSandalen%26gt%3BSandaaltjes probility 0.04
Dames%26gt%3BSchoenen%26gt%3BSneakers%26gt%3BLage+Sneakers probility 0.04
Heren%26gt%3BKleding%26gt%3BJassen%26gt%3BDonsjassen probility 0.02
Heren%26gt%3BKleding%26gt%3BJassen%26gt%3BJacks probility 0.02
Heren%26gt%3BKleding%26gt%3BJassen%26gt%3BLeren+Jass

C:\Users\erm\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                                                                                                    precision    recall  f1-score   support

                                                                                             Dames       0.00      0.00      0.00         0
                                                                          Dames%26gt%3BAccessoires       0.00      0.00      0.00         0
                                                           Dames%26gt%3BAccessoires%26gt%3BBrillen       0.00      0.00      0.00         0
                                                             Dames%26gt%3BAccessoires%26gt%3BEtuis       0.00      0.00      0.00         0
                                                   Dames%26gt%3BAccessoires%26gt%3BHaaraccessoires       0.00      0.00      0.00         0
                                 Dames%26gt%3BAccessoires%26gt%3BHaaraccessoires%26gt%3BHaarbanden       0.06      1.00      0.11         2
                   